In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import csv
import numpy as np

# Veri setini yükle

with open("tsmz.csv", "r", encoding="latin-1") as dosya:
    csv_okuyucu = csv.reader(dosya)
    
    ilk_satir = next(csv_okuyucu)
    ozellik_isimleri = ilk_satir[:-1]
    etiket_isimleri = [ilk_satir[-1]]

    X_verileri = []
    y_verileri = []

    for satir in csv_okuyucu:
        ozellik_degerleri = [float(deger) if deger.replace('.', '', 1).isdigit() else None for deger in satir[:-1]]
        etiket_degeri = float(satir[-1]) if satir[-1].replace('.', '', 1).isdigit() else None

        X_verileri.append(ozellik_degerleri)
        y_verileri.append(etiket_degeri)

# Özellikleri ve etiketleri ayrı değişkenlere ata
X = X_verileri
y = y_verileri

# Metin verilerini vektörlere dönüştür
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=200)

# Veriyi eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = [0 if value is None else value for value in y_train]
y_test = [0 if value is None else value for value in y_test]

# Numpy dizisine dönüştür
y_train = np.array(y_train)
y_test = np.array(y_test)

# Model oluşturma
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=2, batch_size=64)

# Modeli değerlendir
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Modeli kaydet
model.save('tsmz_model.h5')

# Modeli yüklemek için
from keras.models import load_model

# Modeli yüklemek
loaded_model = load_model('tsmz_model.h5')

# Yüklenen modeli değerlendir
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("Loaded model test loss:", score[0])
print("Loaded model test accuracy:", score[1])

# Yeni bir örnek oluşturmak için
new_example = ["Bu bir yeni örnek oluşturmak için"]

# Yeni örneği vektörlere dönüştür
new_example_sequences = tokenizer.texts_to_sequences(new_example)
new_example_padded = pad_sequences(new_example_sequences, maxlen=200)

# Yüklenen modeli kullanarak yeni örneği sınıflandır
prediction = loaded_model.predict(new_example_padded)
print("Prediction:", prediction)

# Prediction sonucunu yorumlamak için
if prediction > 0.5:
    print("The new example is classified as positive")
else:
    print("The new example is classified as negative")